# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML Exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

###### Question 1 : 10 countries with the lowest infant mortality rates
 

###### Question 1 : 10 contries with the lowest infant mortality rates
To demonstrate this We need country name and infant_mortality.The tag infant_mortality is under the country label.From the tree pull out these two and insert into DataFrame.Sort the values in DataFrame then reset the Index and finally view top 10 records 

In [5]:
# Load libraries
from xml.etree import ElementTree as ET
import pandas as pd

# Load data
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

In [6]:
# print names of all countries and their infant mortality rate
data = []
for country in document.iterfind('country'):
    c = country.find('name').text
    infant_mortality = country.find('infant_mortality')
    # take the existing infant_mortality data with respective country and put it in a dataframe
    if(infant_mortality != None):
        infant = float(infant_mortality.text)
        data.append({'Country':c,'Infant_mortality':infant})
    
df = pd.DataFrame(data)
df.head()

,Country,Infant_mortality
0,Albania,13.19
1,Greece,4.78
2,Macedonia,7.90
3,Serbia,6.16
4,Andorra,3.69


In [7]:
#Sort the values then reset the Index and finally view top 10 records
#Generate top 10 countries with lowest Infant_mortality rates
df.sort_values('Infant_mortality').reset_index(drop=True).head(10)

,Country,Infant_mortality
0,Monaco,1.81
1,Japan,2.13
2,Bermuda,2.48
3,Norway,2.48
4,Singapore,2.53
5,Sweden,2.60
6,Czech Republic,2.63
7,Hong Kong,2.73
8,Macao,3.13
9,Iceland,3.15


###### Question 2:  10 cities with the largest population

From the tree pull out city,population and year through iteration and append this into a DataFrame.Sort the values ,reset the index and view top cities

In [8]:
data = []

for country in document.iterfind('country'):
    co = country.find('name').text
    for city in country.iter('city'):
        # including country name for instances where the city name is the same 
        # but located in a different countries
        ci = city.find('name').text + ', ' + co 
        for population in city.iter('population'):
            pop = int(population.text)
            yr = population.attrib['year']
        
        # there are have mulitple years of population numbers for most cities
        # appending the data as part of the city for loop ensures that only 
        # the latest available figures are used
        data.append({'Cities': ci, 'Population': pop, 'Year': yr})   

df = pd.DataFrame(data)
df.head()

,Cities,Population,Year
0,"Tirana, Albania",418495,2011
1,"Shkodër, Albania",77075,2011
2,"Durrës, Albania",113249,2011
3,"Vlorë, Albania",79513,2011
4,"Elbasan, Albania",78703,2011


In [9]:
#Sort the values then reset the Index and finally view top 10 records
City_Population_df = df.loc[:,['Cities','Population']] 
City_Population_df.sort_values('Population', ascending=False).head(10).reset_index(drop=True)


,Cities,Population
0,"Shanghai, China",22315474
1,"Istanbul, Turkey",13710512
2,"Mumbai, India",12442373
3,"Moskva, Russia",11979529
4,"Beijing, China",11716620
5,"São Paulo, Brazil",11152344
6,"Tianjin, China",11090314
7,"Guangzhou, China",11071424
8,"Delhi, India",11034555
9,"Shenzhen, China",10358381


###### Question 3 : 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)


In [10]:
# generate countries and their populations
data = [] 

for country in document.findall('country'):
    for node in list(country):
        if node.tag == 'name':
            co = node.text
        elif node.tag == 'population':
            # the last listed population statistic is used
            pop = int(node.text)
            #meas = node.attrib['measured'] --leads to an error, potentially unpopulated at times
            yr = int(node.attrib['year'])
        elif node.tag == 'ethnicgroup':
            eth = node.text
            perc = float(node.attrib['percentage'])
            epop = int(pop * perc / 100.)
            
            data.append({'Countries':co, 'Country_pop':pop, 'Year':yr,
                        'Ethnicity':eth, 'Country_percentage':perc, 'Population':epop})
    
df = pd.DataFrame(data)
df.head()

,Countries,Country_percentage,Country_pop,Ethnicity,Population,Year
0,Albania,95.0,2800138,Albanian,2660131,2011
1,Albania,3.0,2800138,Greek,84004,2011
2,Greece,93.0,10816286,Greek,10059145,2011
3,Macedonia,64.2,2059794,Macedonian,1322387,2011
4,Macedonia,25.2,2059794,Albanian,519068,2011


In [11]:
# groupby ethinicity with largest population sort values,reset index and finally display top 10 groups
df.groupby('Ethnicity').Population.sum().sort_values(ascending=False).head(10)


Ethnicity
Han Chinese    1245058800
Indo-Aryan      871815583
European        494872201
African         318325104
Dravidian       302713744
Mestizo         157734349
Bengali         146776916
Russian         131856989
Japanese        126534212
Malay           121993548
Name: Population, dtype: int64

###### Question 4 : name and country of a) longest river, b) largest lake and c) airport at highest elevation

** longest river name,country and length**

Check river feature next explore all elements in it.Extract the necessary elements and append it into a DataFrame.Finally sort the values,reset the index and view longest river data.

In [12]:
document.getroot().tag

'mondial'

In [13]:
# generate a unique list of nodes to understand what is available and where 
node_list = []
for node in list(root):
    node_list.append(node.tag)
{node:1 for node in node_list}.keys()


dict_keys(['country', 'continent', 'organization', 'sea', 'river', 'lake', 'island', 'mountain', 'desert', 'airport'])

In [14]:
#pull out country data for mapping country name to code
data = []
for element in document.iterfind('country'):
    if element.find('name') is not None:        
        country = element.find('name').text
        code = element.get('car_code')
        data.append((code,country))
country_mapping = dict(data)

In [15]:
#Check the river feature
elem = document.find('river')
print(ET.tostring(elem))

#Explore river feature elements 
node_list = []
for element in document.findall('river'):
    for node in list(element):
        node_list.append(node.tag)
{node:1 for node in node_list}.keys()

b'<river country="IS" id="river-Thjorsa">\n      <name>Thjorsa</name>\n      <to water="sea-Atlantic" watertype="sea" />\n      <area>7530</area>\n      <length>230</length>\n      <source country="IS">\n         <latitude>65</latitude>\n         <longitude>-18</longitude>\n      </source>\n      <estuary country="IS">\n         <latitude>63.9</latitude>\n         <longitude>-20.8</longitude>\n      </estuary>\n   </river>\n   '


dict_keys(['name', 'to', 'area', 'length', 'source', 'estuary', 'located', 'through'])

In [16]:
# from the river feature extract the necessery elements like river,country and length
data = []

for element in document.iterfind('river'):
    if element.find('name') is not None:        
        river = element.find('name').text
        if element.find('length') is not None:
            length = element.find('length').text
            codes = element.get('country')
            # country attribute may list all relevant country codes separated by space
            for code in codes.split(' '):
                # create list of tuples of river, country, and length
                data.append((river,country_mapping[code],length))

df = pd.DataFrame(data,columns=['River','Country','Length'])
print(df.dtypes)

#Explicit conversion of feature length from object to numeric
df.Length = pd.to_numeric(df.Length, errors='coerce')

# View Longest river name,country and length 
df.sort_values('Length',ascending=False).reset_index(drop=True).head(1)

River      object
Country    object
Length     object
dtype: object


,River,Country,Length
0,Amazonas,Peru,6448.0


**largest lake name,country and area **

Check lake feature next explore all elements in it.Extract the necessary elements and append it into a DataFrame.Finally sort the values,reset the index and view larget lake data.

In [17]:
# Check the lake feature
elem = document.find('lake')
print(ET.tostring(elem))

# Explore lake feature elements 
node_list = []
for element in document.findall('lake'):
    for node in list(element):
        node_list.append(node.tag)
{node:1 for node in node_list}.keys()

b'<lake country="SF" id="lake-Inarisee">\n      <name>Inari</name>\n      <located country="SF" province="lteil-LAP-SF" />\n      <to water="river-Paatsjoki" watertype="river" />\n      <area>1040</area>\n      <latitude>68.95</latitude>\n      <longitude>27.7</longitude>\n      <elevation>119</elevation>\n      <depth>92</depth>\n   </lake>\n   '


dict_keys(['name', 'located', 'to', 'area', 'latitude', 'longitude', 'elevation', 'depth'])

In [18]:
# from the lake feature extract the necessary elements like lake,country and area

data = []

for element in document.iterfind('lake'):
    if element.find('name') is not None:        
        obj = element.find('name').text
        if element.find('area') is not None:
            meas = element.find('area').text
            codes = element.get('country')
            # country attribute may list all relevant country codes separated by space
            for code in codes.split(' '):
                # create list of tuples of lake, country, and area
                data.append((obj,country_mapping[code],meas))

df = pd.DataFrame(data,columns=['Lake','Country','Area'])
print(df.dtypes)

#Explicit conversion of feature area from object to numeric
df.Area = pd.to_numeric(df.Area, errors='coerce')

#View Largest lake name,country and area
df.sort_values('Area',ascending=False).reset_index(drop=True).head(1)


Lake       object
Country    object
Area       object
dtype: object


,Lake,Country,Area
0,Caspian Sea,Russia,386400.0


**name,country of the airport at highest elevation **

Check airport feature next explore all elements in it.Extract the necessary elements and append it into a DataFrame.Finally sort the values,reset the index and view highest elevation airport data.

In [19]:
#Check the airport feature
elem = document.find('airport')
print(ET.tostring(elem))

# Explore the airport feature key elements
node_list = []
for element in document.findall('airport'):
    for node in list(element):
        node_list.append(node.tag)
{node:1 for node in node_list}.keys()

b'<airport city="cty-Afghanistan-2" country="AFG" iatacode="HEA">\n      <name>Herat</name>\n      <latitude>34.210017</latitude>\n      <longitude>62.2283</longitude>\n      <elevation>977</elevation>\n      <gmtOffset>5</gmtOffset>\n   </airport>\n   '


dict_keys(['name', 'latitude', 'longitude', 'elevation', 'gmtOffset', 'located_on'])

In [20]:
# from the airport feature extract the necessary elements like country,airport and elevation
data = [] 

for element in document.findall('airport'):
    # first listed country used since the river can run though multiple countries
    code = element.attrib['country'].split(' ')[0]
    for node in list(element):
        if node.tag == 'name':
            airport = node.text
        elif node.tag == 'elevation':
            if node.text is not None:
                elevation = int(node.text)
            # Generate list of airports and their elevation
            data.append({'Country_code': code, 'Airport':airport, 'Elevation':elevation})
    
airports = pd.DataFrame(data)

airports.sort_values('Elevation', ascending=False).reset_index(drop=True).head(1)

,Airport,Country_code,Elevation
0,El Alto Intl,BOL,4063
